In [1]:
import nltk
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [2]:
import statistics

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
latlib_df = pd.read_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data/latinlibrary_data.pkl")

In [5]:
ne_df = pd.read_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data/latinlibrary_ne_data.pkl")

In [6]:
#here's some of the data:
latlib_df.head()

,title,text,text_raw,tokens,token_count
0,Ammianus: Liber XXXI,AMMIANI MARCELLINI HISTORIAE LIBER XXXI ...,"[AMMIANI MARCELLINI HISTORIAE LIBER XXXI\n, \n...","[AMMIANI, MARCELLINI, HISTORIAE, LIBER, XXXI, ...",11030
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524
2,Valerius Maximus VIII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10337
3,Valerius Maximus IV,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10852
4,Valerius Maximus VII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",8918


In [7]:
ne_df.head()

,name,era
0,AMMIANVS MARCELLINVS,(4th century A.D.)
1,VALERIVS MAXIMVS,(1st century A.D.)
2,GAIVS VALERIVS FLACCVS,(died c. 90 A.D.)
4,P. VERGILIVS MARO,(70 – 19 B.C.)
6,ALBIVS TIBVLLVS,(c. 54 – 19 B.C.)


Here's some basic info on the data contained in latlib_df:

In [8]:
#overall shape of the dataframe:
latlib_df.shape

(672, 5)

In [9]:
latlib_df.describe()

,token_count
count,672.000000
mean,6331.989583
std,5564.159005
min,0.000000
25%,1681.500000
50%,5578.500000
75%,8590.500000
max,38414.000000


In [10]:
latlib_df.info()
#good! no null objects

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        672 non-null    object
 1   text         672 non-null    object
 2   text_raw     672 non-null    object
 3   tokens       672 non-null    object
 4   token_count  672 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 26.4+ KB


let's work on seperating works by style (prose or poetry):

In [11]:
#use Aeneid (P. VERGILI MARONIS AENEIDOS) + Ovid's Metamorphoses (P. OVIDI NASONIS METAMORPHOSEON) for verse examples

In [12]:
metamorphoses_ovid = latlib_df[latlib_df['text'].str.contains('P. OVIDI NASONIS METAMORPHOSEON')]

In [13]:
aeneid = latlib_df[latlib_df['text'].str.contains('P. VERGILI MARONIS AENEIDOS')]

In [14]:
aeneid

,title,text,text_raw,tokens,token_count
123,P. VERGILI MARONIS AENEIDOS LIBER DVODECIMVS,P. VERGILI MARONIS AENEIDOS LIBER DVODECIMVS ...,[P. VERGILI MARONIS AENEIDOS LIBER DVODECIMVS\...,"[P., VERGILI, MARONIS, AENEIDOS, LIBER, DVODEC...",7262
124,P. VERGILI MARONIS AENEIDOS LIBER VNDECIMVS,P. VERGILI MARONIS AENEIDOS LIBER VNDECIMVS O...,[P. VERGILI MARONIS AENEIDOS LIBER VNDECIMVS\n...,"[P., VERGILI, MARONIS, AENEIDOS, LIBER, VNDECI...",6884
125,P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS,P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS Ut ...,[P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS\n\n...,"[P., VERGILI, MARONIS, AENEIDOS, LIBER, OCTAVV...",5420
126,Aeneid IV,P. VERGILI MARONIS AENEIDOS LIBER QVARTVS At ...,"[P. VERGILI MARONIS AENEIDOS LIBER QVARTVS\n, ...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, QVARTV...",5408
127,P. VERGILI MARONIS AENEIDOS LIBER DECIMVS,P. VERGILI MARONIS AENEIDOS LIBER DECIMVS Pan...,"[P. VERGILI MARONIS AENEIDOS LIBER DECIMVS\n, ...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, DECIMV...",6985
128,P. VERGILI MARONIS AENEIDOS LIBER SEPTIMVS,P. VERGILI MARONIS AENEIDOS LIBER SEPTIMVS Tu...,"[P. VERGILI MARONIS AENEIDOS LIBER SEPTIMVS\n,...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, SEPTIM...",6058
129,P. VERGILI MARONIS AENEIDOS LIBER NONVS,P. VERGILI MARONIS AENEIDOS LIBER NONVS Atque...,"[P. VERGILI MARONIS AENEIDOS LIBER NONVS\n, \n...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, NONVS,...",6222
130,P. VERGILI MARONIS AENEIDOS LIBER TERTIVS,P. VERGILI MARONIS AENEIDOS LIBER TERTIVS Pos...,"[P. VERGILI MARONIS AENEIDOS LIBER TERTIVS\n, ...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, TERTIV...",5318
131,P. VERGILI MARONIS AENEIDOS LIBER SECVNDVS,P. VERGILI MARONIS AENEIDOS LIBER SECVNDVS Co...,"[P. VERGILI MARONIS AENEIDOS LIBER SECVNDVS\n,...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, SECVND...",6069
132,P. VERGILI MARONIS AENEIDOS LIBER SEXTVS,P. VERGILI MARONIS AENEIDOS LIBER SEXTVS Sic ...,"[P. VERGILI MARONIS AENEIDOS LIBER SEXTVS\n, \...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, SEXTVS...",6944


In [15]:
[len(x) for x in aeneid.text_raw]
#this gets me how many lines per-work there are 

[964, 924, 741, 715, 920, 833, 825, 727, 811, 905, 767, 880]

In [16]:
count = 0
for i in aeneid.text_raw:
    count += len(i)
print(count) #this is how many lines are in the aeneid in total

10012


In [17]:
def MaxLineLength(l):
    return(len(max(l, key=len)))

In [18]:
max(aeneid.text_raw.map(MaxLineLength))
#so the longest line is 87 characters long, that's my threshold for a verse line (depending on Ovid I guess)

87

In [19]:
max(metamorphoses_ovid.text_raw.map(MaxLineLength)) #yep, 87 it is!

81

now to actually use that number to quantify prose v. verse!

In [20]:
latlib_df['max_line_length'] = latlib_df.text_raw.map(MaxLineLength)

In [21]:
latlib_df.head(5)

,title,text,text_raw,tokens,token_count,max_line_length
0,Ammianus: Liber XXXI,AMMIANI MARCELLINI HISTORIAE LIBER XXXI ...,"[AMMIANI MARCELLINI HISTORIAE LIBER XXXI\n, \n...","[AMMIANI, MARCELLINI, HISTORIAE, LIBER, XXXI, ...",11030,774
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524,2080
2,Valerius Maximus VIII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10337,1308
3,Valerius Maximus IV,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10852,2034
4,Valerius Maximus VII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",8918,2141


In [22]:
latlib_df.drop(latlib_df.index[latlib_df['token_count']<=10], inplace=True)


In [23]:
def Poems(l):
    if l <= 87:
        return('verse')
    else:
        return('prose')


In [24]:
latlib_df['style'] = latlib_df.max_line_length.map(Poems)

## below I'm going to add the aspects of ne_df to latlib_df for easier analysis:

In [25]:
ne_df['name'] = [w[:-2] for w in ne_df.name]

In [26]:
def MatchName(l):
    for name in ne_df.name:
        last_word = name.split()[len(name.split()) - 1]
        if last_word in l:
            return name
            return "None"

In [27]:
latlib_df['author'] = latlib_df.text.map(MatchName)

to put the eras in the main df it's not going to be elegant, but I want to move on with my analysis so I'm going to merge the info into a new column for an easier method:

In [28]:
ne_df['merged'] = ne_df['era']+ne_df['name']

In [29]:
def MatchEra(l):
    for x in ne_df.merged:
        last_word = x.split()[len(x.split()) - 1]
        if last_word in l:
            clean = re.compile(r'\)\w* \w* \w*|\)\w* \w*|\)\w+\. \w* \w*|\)\w+|\(')
            return(clean.sub(' ', x))
        

In [30]:
latlib_df['life_span'] = latlib_df.text.map(MatchEra)

In [31]:
latlib_df = latlib_df.fillna('none') #just in case, I think there's some Suetonius that's wonky

That wasn't very pretty, but it did the trick and now the df itself looks nice!

In [32]:
latlib_df.sample(5)

,title,text,text_raw,tokens,token_count,max_line_length,style,author,life_span
180,Justin XIVL,HISTORIARUM PHILIPPICARUM IN EPITOMEN REDACTI ...,[HISTORIARUM PHILIPPICARUM IN EPITOMEN REDACTI...,"[HISTORIARUM, PHILIPPICARUM, IN, EPITOMEN, RED...",1394,337,prose,TITVS LVCRETIVS CAR,c. 94 – c. 49 B.C.
178,Petronius: Satiricon,"C. PETRONII SATIRICON LIBER <. . .> [I] ""Num ...","[C. PETRONII SATIRICON LIBER\n, \n\n<. . .> \n...","[C., PETRONII, SATIRICON, LIBER, <, ., ., ., >...",38414,3177,prose,P. VERGILIVS MA,70 – 19 B.C.
445,Quintilian: Declamatio Maior II,M. FABI QVINTILIANI DECLAMATIO MAIOR SECVNDA ...,[M. FABI QVINTILIANI DECLAMATIO MAIOR SECVNDA\...,"[M., FABI, QVINTILIANI, DECLAMATIO, MAIOR, SEC...",5526,2944,prose,P. VERGILIVS MA,70 – 19 B.C.
646,Curtius Rufus: Historiae Alexandri Magni VII,QUINTI CURTI RUFI HISTORIAE ALEXANDRI MAGNI LI...,[QUINTI CURTI RUFI HISTORIAE ALEXANDRI MAGNI L...,"[QUINTI, CURTI, RUFI, HISTORIAE, ALEXANDRI, MA...",11576,5566,prose,P. VERGILIVS MA,70 – 19 B.C.
289,Livy: Book XXXV,TITI LIVI AB VRBE CONDITA LIBER XXXV ...,"[TITI LIVI AB VRBE CONDITA LIBER XXXV\n\n, \t,...","[TITI, LIVI, AB, VRBE, CONDITA, LIBER, XXXV, [...",14579,3054,prose,T. LIVI,59 B.C. – A.D. 17 . LIVI


let's split life spans into actual eras (Golden Age, Silver Age, Later):

In [57]:
def SortEra(l):
    if 'B.C.' in l and 'A.D.' not in l:
        return('Golden Age')
    elif 'A.D.' in l and 'B.C.' not in l:
        if 'century' in l:
            if '3' in l or '4' in l:
                return('Later')
            elif '1' in l or '2' in l:
                return('Silver Age')
        #elif 'century' not in l:
            

        #return('silver and later')
    else:
        return('none')

In [58]:
latlib_df['era'] = latlib_df.life_span.map(SortEra)

In [59]:
latlib_df.sample(5)

,title,text,text_raw,tokens,token_count,max_line_length,style,author,life_span,term,era
669,Auli Gellii Noctes Atticae: Liber III,AVLI GELLI NOCTES ATTICAE: LIBER III ...,"[AVLI GELLI NOCTES ATTICAE: LIBER III\n, , ...","[AVLI, GELLI, NOCTES, ATTICAE, :, LIBER, III, ...",6822,837,prose,AVLVS GELLI,c. A.D. 125 – after 180,,None
131,P. VERGILI MARONIS AENEIDOS LIBER SECVNDVS,P. VERGILI MARONIS AENEIDOS LIBER SECVNDVS Co...,"[P. VERGILI MARONIS AENEIDOS LIBER SECVNDVS\n,...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, SECVND...",6069,85,verse,P. VERGILIVS MA,70 – 19 B.C.,,Golden Age
210,Justin XVIII,HISTORIARUM PHILIPPICARUM IN EPITOMEN REDACTI ...,[HISTORIARUM PHILIPPICARUM IN EPITOMEN REDACTI...,"[HISTORIARUM, PHILIPPICARUM, IN, EPITOMEN, RED...",2311,314,prose,TITVS LVCRETIVS CAR,c. 94 – c. 49 B.C.,,Golden Age
363,Ovid: Heroides IV,P. OVIDI NASONIS EPISTVLAE HEROIDVM Quam nis...,"[P. OVIDI NASONIS EPISTVLAE HEROIDVM\n, \n, \n...","[P., OVIDI, NASONIS, EPISTVLAE, HEROIDVM, Quam...",1349,70,verse,P. OVIDIVS NA,43 B.C. – 17 A.D.,,none
476,"Silius, Liber VI","SILIUS ITALICUS PUNICA LIBER SEXTUS Iam, Tarte...","[SILIUS ITALICUS, PUNICA LIBER SEXTUS\n, \nIam...","[SILIUS, ITALICUS, PUNICA, LIBER, SEXTUS, Iam,...",5285,67,verse,SILIVS ITALIC,25/6 – 101 A.D.,,None


In [60]:
for x in ne_df.era:
    if 'A.D.' in x and 'B.C.' not in x:
        if 'century' not in x:
            if 
                print(x)


SyntaxError: invalid syntax (<ipython-input-60-261c7c863059>, line 4)

In [ ]:
"""
the only 3 non-century AD values that are Later
(c. A.D. 125 – after 180)
(c. 370 – c. 404 A.D.)
(c. A.D. 123/5 – c. 180)
if the value directly preceding the - contains 3 characters?
"""

## sword word analysis:

In [33]:
latlib_df['term'] = ''

In [34]:
#gladius:
gladius = latlib_df[(latlib_df.text.str.contains(pat = 'gladius', case=False)) | (latlib_df.text.str.contains(pat = 'gladii', case=False)) | (latlib_df.text.str.contains(pat = 'gladi', case=False)) | (latlib_df.text.str.contains(pat = 'gladio', case=False)) | (latlib_df.text.str.contains(pat = 'gladium', case=False)) | (latlib_df.text.str.contains(pat = 'gladie', case=False)) | (latlib_df.text.str.contains(pat = 'gladiorum', case=False)) | (latlib_df.text.str.contains(pat = 'gladiis', case=False)) | (latlib_df.text.str.contains(pat = 'gladios', case=False))]

In [35]:
#ferrum:
ferrum = latlib_df[(latlib_df.text.str.contains(pat = 'ferrum', case=False)) | (latlib_df.text.str.contains(pat = 'ferri', case=False)) | (latlib_df.text.str.contains(pat = 'ferro', case=False)) | (latlib_df.text.str.contains(pat = 'ferra', case=False)) | (latlib_df.text.str.contains(pat = 'ferrorum', case=False)) | (latlib_df.text.str.contains(pat = 'ferris', case=False))]

In [36]:
#mucro:
mucro = latlib_df[(latlib_df.text.str.contains(pat = 'mucro', case=False)) | (latlib_df.text.str.contains(pat = 'mucronis', case=False)) | (latlib_df.text.str.contains(pat = 'mucroni', case=False)) | (latlib_df.text.str.contains(pat = 'mucronem', case=False)) | (latlib_df.text.str.contains(pat = 'mucrone', case=False)) | (latlib_df.text.str.contains(pat = 'mucrones', case=False)) | (latlib_df.text.str.contains(pat = 'mucronum', case=False)) | (latlib_df.text.str.contains(pat = 'mucronibus', case=False))]

In [37]:
#ensis:
ensis = latlib_df[(latlib_df.text.str.contains(pat = 'ensis', case=False)) | (latlib_df.text.str.contains(pat = 'ensi', case=False)) | (latlib_df.text.str.contains(pat = 'ensem', case=False)) | (latlib_df.text.str.contains(pat = 'ense', case=False)) | (latlib_df.text.str.contains(pat = 'enses', case=False)) | (latlib_df.text.str.contains(pat = 'ensium', case=False)) | (latlib_df.text.str.contains(pat = 'ensibus', case=False))]

In [38]:
#capulus:
capulus = latlib_df[(latlib_df.text.str.contains(pat = 'capulus', case=False)) | (latlib_df.text.str.contains(pat = 'capuli', case=False)) | (latlib_df.text.str.contains(pat = 'capulo', case=False)) | (latlib_df.text.str.contains(pat = 'capulum', case=False)) | (latlib_df.text.str.contains(pat = 'capulorum', case=False)) | (latlib_df.text.str.contains(pat = 'capulis', case=False)) | (latlib_df.text.str.contains(pat = 'capulos', case=False))]

In [39]:
#let's see how many occurences of each word there are:
print('gladius: ',len(gladius))
print('ferrum: ',len(ferrum))
print('mucro: ',len(mucro))
print('ensis: ',len(ensis))
print('capulus: ',len(capulus)) #'capulus' is the least common, only appearing 48 times, let's look at that data!

gladius:  277
ferrum:  462
mucro:  89
ensis:  573
capulus:  48


In [40]:
capulus.head()

,title,text,text_raw,tokens,token_count,max_line_length,style,author,life_span,term
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524,2080,prose,VALERIVS MAXIM,1st century A.D.,
25,Statius: Achilleid II,PVBLIVS PAPINIVS STATIVS ACHILLEID II Exuit im...,"[PVBLIVS PAPINIVS STATIVS, \nACHILLEID II\n, \...","[PVBLIVS, PAPINIVS, STATIVS, ACHILLEID, II, Ex...",1243,71,verse,PVBLIVS PAPINIVS STATI,c. 45 – 96 A.D.,
31,Statius: Thebaid XI,PVBLIVS PAPINIVS STATIVS THEBAID XI Postquam m...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID XI\n, \nP...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, XI, Post...",6017,72,verse,PVBLIVS PAPINIVS STATI,c. 45 – 96 A.D.,
34,Statius: Thebaid XII,PVBLIVS PAPINIVS STATIVS THEBAID XII Nondum cu...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID XII\n, \n...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, XII, Non...",6244,72,verse,PVBLIVS PAPINIVS STATI,c. 45 – 96 A.D.,
35,Statius: Thebaid IX,PVBLIVS PAPINIVS STATIVS THEBAID IX Asperat Ao...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID IX\n, \nA...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, IX, Aspe...",6925,74,verse,PVBLIVS PAPINIVS STATI,c. 45 – 96 A.D.,


Just from looking at these titles there's a pretty fair distribution of genre between the works (some philosophy, epic poetry, history, and even comedy plays!)

In [41]:
print(len(gladius[gladius['style']=='prose'])/len(gladius)*100)
print(len(gladius[gladius['style']=='verse'])/len(gladius)*100)
#the results here are skewed highly in favor of prose

79.78339350180505
20.216606498194945


In [42]:
print(len(ferrum[ferrum['style']=='prose'])/len(ferrum)*100)
print(len(ferrum[ferrum['style']=='verse'])/len(ferrum)*100)
#here there's more in the prose category

66.88311688311688
33.116883116883116


In [43]:
print(len(mucro[mucro['style']=='prose'])/len(mucro)*100)
print(len(mucro[mucro['style']=='verse'])/len(mucro)*100)
#it's closer here, but still in favor of prose

65.1685393258427
34.831460674157306


In [44]:
print(len(ensis[ensis['style']=='prose'])/len(ensis)*100)
print(len(ensis[ensis['style']=='verse'])/len(ensis)*100)
#heavily in favor of prose

70.85514834205934
29.144851657940663


In [45]:
print(len(capulus[capulus['style']=='prose'])/len(capulus)*100)
print(len(capulus[capulus['style']=='verse'])/len(capulus)*100)
#interestingly enough, verse is more popular than prose here

39.58333333333333
60.416666666666664


because the results are overwhelmingly in favor of prose in most categories let's look at how they match up in latlib_df

In [46]:
print(len(latlib_df[latlib_df['style']=='prose']))
print(len(latlib_df[latlib_df['style']=='verse']))
#so prose definitely shows more than verse, which is not super surprising

425
185


In [47]:
p_n = len(latlib_df[latlib_df['style']=='prose'])/len(latlib_df)*100
v_n = len(latlib_df[latlib_df['style']=='verse'])/len(latlib_df)*100
print('percentage of prose works in data: ',p_n,'%')
print('percentage of verse works in data: ',v_n,'%')

percentage of prose works in data:  69.67213114754098 %
percentage of verse works in data:  30.327868852459016 %


In [48]:
gladius.term = ['gladius' for x in gladius.term]
ferrum.term = ['ferrum' for x in ferrum.term]
mucro.term = ['mucro' for x in mucro.term]
ensis.term = ['ensis' for x in ensis.term]
capulus.term = ['capulus' for x in capulus.term]

/Users/francesharrington/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [49]:
ensis.head(5)

,title,text,text_raw,tokens,token_count,max_line_length,style,author,life_span,term
0,Ammianus: Liber XXXI,AMMIANI MARCELLINI HISTORIAE LIBER XXXI ...,"[AMMIANI MARCELLINI HISTORIAE LIBER XXXI\n, \n...","[AMMIANI, MARCELLINI, HISTORIAE, LIBER, XXXI, ...",11030,774,prose,AMMIANVS MARCELLIN,4th century A.D.,ensis
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524,2080,prose,VALERIVS MAXIM,1st century A.D.,ensis
2,Valerius Maximus VIII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10337,1308,prose,VALERIVS MAXIM,1st century A.D.,ensis
3,Valerius Maximus IV,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10852,2034,prose,VALERIVS MAXIM,1st century A.D.,ensis
4,Valerius Maximus VII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",8918,2141,prose,VALERIVS MAXIM,1st century A.D.,ensis


In [50]:
df_swords = pd.concat([gladius, ferrum, mucro, ensis, capulus])

In [51]:
df_swords.sample(5)

,title,text,text_raw,tokens,token_count,max_line_length,style,author,life_span,term
442,Quintilian: Declamatio Maior VI,M. FABI QVINTILIANI DECLAMATIO MAIOR SEXTVS Qu...,[M. FABI QVINTILIANI DECLAMATIO MAIOR SEXTVS\n...,"[M., FABI, QVINTILIANI, DECLAMATIO, MAIOR, SEX...",6266,2419,prose,P. VERGILIVS MA,70 – 19 B.C.,ferrum
360,Ovid: Heroides IX,P. OVIDI NASONIS EPISTVLAE HEROIDVM Mittor a...,"[P. OVIDI NASONIS EPISTVLAE HEROIDVM\n, \n, \n...","[P., OVIDI, NASONIS, EPISTVLAE, HEROIDVM, Mitt...",1246,52,verse,P. OVIDIVS NA,43 B.C. – 17 A.D.,ensis
415,Pliny the Younger,C. PLINII CAECILII SECVNDI EPISTVLARVM LIBER O...,[C. PLINII CAECILII SECVNDI EPISTVLARVM LIBER ...,"[C., PLINII, CAECILII, SECVNDI, EPISTVLARVM, L...",7344,567,prose,AMMIANVS MARCELLIN,4th century A.D.,ferrum
401,Plautus: Casina,T. MACCI PLAVTI CASINA OLYMPIO VILICVS CHALI...,"[T. MACCI PLAVTI CASINA\n, \n, \n, \nOLYMPIO V...","[T., MACCI, PLAVTI, CASINA, OLYMPIO, VILICVS, ...",9591,84,verse,P. VERGILIVS MA,70 – 19 B.C.,capulus
461,Sallust: Speech of Macer,"ORATIO MACRI TRIB. PLEB. AD PLEBEM Si, Quirit...","[ORATIO MACRI TRIB. PLEB. AD PLEBEM\n, \n Si, ...","[ORATIO, MACRI, TRIB, ., PLEB, ., AD, PLEBEM, ...",1000,1656,prose,P. VERGILIVS MA,70 – 19 B.C.,ensis


In [52]:
print('gladius era variance: ', len(set([x for x in gladius['life_span']])))
print('ferrum era variance: ',len(set([x for x in ferrum['life_span']])))
print('mucro era variance: ',len(set([x for x in mucro['life_span']])))
print('ensis era variance: ',len(set([x for x in ensis['life_span']])))
print('capulus era variance: ',len(set([x for x in capulus['life_span']]))) #capulus has the least amount of era variance, lets take a look!

gladius era variance:  22
ferrum era variance:  26
mucro era variance:  16
ensis era variance:  27
capulus era variance:  12


In [53]:
(set([x for x in capulus['life_span']])) #pretty large span of time too, from the 2nd century BC to the 4th century AD!
#however, the majority of usage comes from after the year 0

{' 106 – 43 B.C. ',
 ' 116 – 27 B.C. ',
 ' 1st century A.D. ',
 ' 25/6 – 101 A.D. ',
 ' 43 B.C. – 17 A.D. ',
 ' 4th century A.D. ',
 ' 63 – c. 113 A.D.  SECVND',
 ' 70 – 19 B.C. ',
 ' c. 4 B.C. – A.D. 65 ',
 ' c. 45 – 96 A.D. ',
 ' c. 56 – c. 117 A.D. ',
 'none'}

In [54]:
#lets look through author usages now:
print(set(df_swords.author))#all the authors that use a sword word
print(len(set(df_swords.author))) #that would be all 31, according to the set difference check below

{'PVBLIVS PAPINIVS STATI', 'SCRIPTORES HISTORIAE AVGVST', 'P. OVIDIVS NA', 'M. PORCIVS CATO CENS', 'C. PETRONI', 'SEXTVS PROPERTI', 'P. CORNELIVS TACIT', 'C. PLINIVS CAECILIVS SECVND', 'SILIVS ITALIC', 'C. IULIUS CAES', 'P. TERENTIVS AF', 'C. SVETONIVS TRANQVILL', 'L. ANNAEVS SENE', 'ALBIVS TIBVLL', 'PHAEDR', 'AMMIANVS MARCELLIN', 'M. VITRVVIVS POLL', 'FLOR', 'M. TERENTIVS VAR', 'CORNELIVS NEP', 'AVLVS GELLI', 'C. SALLVSTIVS CRISP', 'T. LIVI', 'VALERIVS MAXIM', 'M. FABIVS QVINTILIAN', 'EVTROPI', 'TITVS LVCRETIVS CAR', 'P. VERGILIVS MA', 'none', 'M. TVLLIVS CICE', 'GAIVS VALERIVS FLACC'}
31


In [55]:
x = set(df_swords.author)
y = set(latlib_df.author)
z = x.difference(y)
print(z) #yep, every author uses a sword word in at least one work!

set()


final stat overview & save updated examples out:

In [ ]:
latlib_df.describe()

In [ ]:
#updated data samples so they reflect my current data:
examples = latlib_df.head(10)
examples.to_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data_samples/examples.pkl")